In [ ]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

입력과 결과로 Train 데이터 나누기 + 인코딩

In [ ]:
# 데이터 준비
df1 = pd.read_csv('/content/drive/MyDrive/파이널프로젝트 파일/merged_all_train.csv')
# 입력과 결과로 나눈다.
X = df1.drop(['Segment','ID','기준년월'], axis=1)
y = df1['Segment']
# 문자열 -> 숫자
encoder1 = LabelEncoder()
encoder1.fit(y)
y2 = encoder1.transform(y)


In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd

# 1. 범주형 → 수치형(Label Encoding)
label_enconders = {}
X_encoded = X.copy()
for col in X_encoded.columns:
    if X_encoded[col].dtype == 'object' or X_encoded[col].dtype.name == 'category':
        le = LabelEncoder()
        X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))
        label_enconders[col] = le

# 2. 표준화
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)
feature_cols = X.columns.tolist()

In [ ]:
# 학습용과 검증용으로 나눈다.
train_X = X_scaled
train_y = y2

학습

In [ ]:
model1 = RandomForestClassifier()
# 모델 학습
model1.fit(train_X, train_y)

RandomForestClassifier()

예측 후 파일 저장

In [ ]:
# 데이터 준비
df2 = pd.read_csv('/content/drive/MyDrive/파이널프로젝트 파일/merged_all_test.csv')

In [ ]:
# 2) 모델 학습 때 사용한 피처만 남기기
X_test = df2.drop(['ID', '기준년월', 'Segment'], axis=1, errors='ignore')

# 1. 범주형 → 수치형(Label Encoding)
X_encoded = X_test.copy()
for col,le in label_enconders.items():
    if col in X_encoded.columns :
        X_encoded[col] = le.transform(X_encoded[col].astype(str))

# 학습 단계의 feature_cols 순서 그대로 재정렬
X_encoded = X_encoded.reindex(columns=feature_cols)
# 3) 스케일러로 변환
X_test_scaled = scaler.transform(X_encoded)

# 4) 예측
y_test_pred = model1.predict(X_test_scaled)
# 결과데이터를 복원한다.
result = encoder1.inverse_transform(y_test_pred)
# 5) 복원 & 저장
df2['Segment_pred'] = result
df2.to_csv('/content/drive/MyDrive/파이널프로젝트 파일/RF_predictions.csv', index=False, encoding='utf-8-sig')

In [ ]:
df2['Segment_pred']

,Segment_pred
0,E
1,E
2,E
3,E
4,E
...,...
599995,E
599996,E
599997,E
599998,E


statified kfold 교차검증으로 f1_micro 및 accuracy score 계산(시간이 오래 걸려서 밑으로 빼놓았음.)

In [ ]:
model1 = RandomForestClassifier()

#Stratified K-Fold 교차검증
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
r1 = cross_val_score(model1, train_X, train_y, scoring='f1_micro', cv=skf)

print(f'평균 f1_micro Score : {r1.mean()}')

평균 f1_micro Score : 0.8779595833333333


In [ ]:
# Accuracy
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
acc = cross_val_score(model1, train_X, train_y, scoring='accuracy', cv=skkfold)
print(f'평균 Accuracy Score : {acc.mean()}')

평균 Accuracy Score : 0.8777741666666665
